Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [ ]:
df.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [ ]:
df= df.astype('int')

In [ ]:
df.shape

(5000, 5)

## 2. Ответ на вопрос.

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

## 3. Алгоритм преобразования

**Алгоритм**

Для шифрования данных матрицицу признаков $X$ мы умножам на матрицу со случайными значениями с нормальным распределением $P$. После этой операции мы получаем матрицу $Z$ с измененными значениями. 

***Условие работы алгоритма:*** матрица $P$ должна быть обратимой, т.е. $P P^{-1} = E$

**_Формула шифрования:_** $$ X P=Z$$


Для дешифровки данных необходимо измененную матриц $Z$ умножить на обратную случайной матрице $P$ матрицу $P^{-1}$

**_Формула дешифровки:_**
$$Z P^{-1}=X$$

**Обоснование**

Как видно из пункта 2 предсказание не изменится, так как $a = X w = X P w' = E y$ следовательно, предсказание $a$ не зависит от значенияй в матрице $X$

## 4. Проверка алгоритма

Проверим значение $R2$ до преобразования.

In [ ]:
features = df.drop('Страховые выплаты', axis=1).values
target = df['Страховые выплаты'].values

model = LinearRegression()
r2 = cross_val_score(model, features, target, cv=5).mean()
print('R2 =', r2)

R2 = 0.4231137692077528


Проверим значение $R2$ после преобразования.

In [ ]:
# Создадим матрицу со случайными значениями по нормальному распределению.
A = np.random.randn(features.shape[1], features.shape[1])
print('Матрица со случайными значениями')
print(A)
print()

#Проверим матрицу на обратимость
A_inv = np.linalg.inv(A)
print('Обратная матрица')
print(A_inv)

print()
print('Проверка матрицы на обратимость')
print(A @ A_inv)

Матрица со случайными значениями
[[-1.8804297  -0.44653018  0.32859715  0.99510161]
 [ 0.26646812 -1.18935696 -0.01485983 -0.60196705]
 [ 0.82192     1.1251349   0.87921424 -0.42670491]
 [-1.00327629  0.31298776  1.65635722  1.1002961 ]]

Обратная матрица
[[-1.40758525 -0.18390024 -0.97407581  0.79464567]
 [ 0.43151666 -0.51559853  0.63367868 -0.42659648]
 [ 0.04690423  0.47470896  0.41518418  0.37830345]
 [-1.47682655 -0.7356332  -1.69344794  1.18528366]]

Проверка матрицы на обратимость
[[ 1.00000000e+00 -2.17659585e-17  1.91100050e-16 -2.21164037e-16]
 [ 3.30363602e-17  1.00000000e+00 -1.08502580e-16 -1.37840854e-17]
 [-1.36745650e-17 -3.07058794e-17  1.00000000e+00 -1.30720998e-16]
 [ 8.22324975e-17 -1.93763731e-16 -7.53963300e-17  1.00000000e+00]]


In [ ]:
features_A = df.drop('Страховые выплаты', axis=1).values @ A
target = df['Страховые выплаты'].values

model_1 = LinearRegression()
r2_new = cross_val_score(model_1, features_A, target, cv=5).mean()
print('R2 =', r2_new)

R2 = 0.42311376920772564


**Вывод**

Значения $R2$ до преобразования и после одинаковы. Исходя из этого можем сделать вывод, что наш алгоритм шифровки работает и не влияет на качество модели.